In [1]:
! nvidia-smi

Tue Jul  8 23:09:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:52:00.0 Off |                    0 |
| N/A   29C    P0             45W /  270W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
import argparse
import os
import time
from typing import Dict, Tuple, Union, Optional, Callable, List, Any
from torch.utils.data import Dataset, DataLoader, Subset
import numpy as np
import torch
import torch.distributed as dist
import transformers
import yaml
from datasets import (
    Dataset,
    load_dataset,
    DatasetDict,
    IterableDatasetDict,
    IterableDataset,
)
from datasets import Dataset as HFDataset, DatasetDict
from sklearn.metrics import f1_score, matthews_corrcoef
from sklearn.model_selection import KFold
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    PreTrainedTokenizer,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    DataCollatorWithPadding,
    PreTrainedModel,
    AutoConfig,
)




/home/wenduoc/mambaforge/envs/dnalongbench/lib/python3.9/site-packages/transformers/utils/generic.py:496: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/wenduoc/mambaforge/envs/dnalongbench/lib/python3.9/site-packages/transformers/utils/generic.py:353: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/wenduoc/mambaforge/envs/dnalongbench/lib/python3.9/site-packages/transformers/utils/generic.py:353: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/wenduoc/mambaforge/envs/dnalongbench/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/h

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device 

device(type='cuda')

In [4]:
root = '/work/magroup/shared/DNA_LLM/DNALongBench/'

In [5]:
import torch
import dnalongbench
from dnalongbench.utils import load_data, BasenjiDataSet
import numpy as np
import tensorflow as tf

In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
tf.config.set_visible_devices([], 'GPU')

In [8]:
# human_fasta_path = root + 'regulatory_sequence_activity_prediction/human/seqs/hg38.ml.fa'
# data_path = root + 'regulatory_sequence_activity_prediction/'
# organism = 'human'

In [9]:
# train_dataset = BasenjiDataSet(data_path, organism, 'train', 196608, human_fasta_path)

In [10]:
# def custom_collate(batch):
#     x, y = zip(*batch)
#     x = torch.tensor(np.stack(x), dtype=torch.float32)
#     y = torch.tensor(np.stack(y), dtype=torch.float32)
#     return x, y

# train_loader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=4, num_workers=0, collate_fn=custom_collate
# )

In [11]:
# for batch in train_loader: 
#         x, y = batch
#         print('x:',x.size())
#         print('y:',y.size())
#         break


In [12]:
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, num_workers=0)

In [13]:
def load_data(root='/work/magroup/shared/DNA_LLM/DNALongBench/', task_name = 'regulatory_sequence_activity', organism = 'human', cell_type=None, batch_size=16, sequence_length=196608):
    if task_name == 'regulatory_sequence_activity':
        assert organism == "human" or organism == "mouse"
        human_fasta_path = root + 'regulatory_sequence_activity_prediction/human/seqs/hg38.ml.fa'
        mouse_fasta_path = root + 'regulatory_sequence_activity_prediction/mouse/seqs/mm10.ml.fa'
        data_path = root + 'regulatory_sequence_activity_prediction/'
        
        # SEQUENCE_LENGTH = 196608
        # BIN_SIZE = 128
        # TARGET_LENGTH = 896

        fasta_path = human_fasta_path if organism == "human" else mouse_fasta_path

        train_dataset = BasenjiDataSet(data_path, organism, 'train', sequence_length, fasta_path)
        valid_dataset = BasenjiDataSet(data_path, organism, 'valid', sequence_length, fasta_path)
        test_dataset = BasenjiDataSet(data_path, organism, 'test', sequence_length, fasta_path)

        def custom_collate(batch):
            x, y = zip(*batch)
            x = torch.tensor(np.stack(x), dtype=torch.float32)
            y = torch.tensor(np.stack(y), dtype=torch.float32)
            return x, y
        
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=0, collate_fn=custom_collate)
        valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, num_workers=0, collate_fn=custom_collate)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=0, collate_fn=custom_collate)
        return train_loader, valid_loader, test_loader 

In [14]:
train_loader, valid_loader, test_loader = load_data(root=root, task_name = 'regulatory_sequence_activity', organism = 'human', cell_type=None, batch_size=16, sequence_length=196608)

In [15]:
for batch in train_loader: 
        x, y = batch
        print('x:',x.size())
        print('y:',y.size())
        break


2025-07-08 23:09:43.563517: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


x: torch.Size([16, 196608, 4])
y: torch.Size([16, 896, 5313])


In [16]:
!nvidia-smi

Tue Jul  8 23:09:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:52:00.0 Off |                    0 |
| N/A   29C    P0             45W /  270W |       4MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [17]:
def collate_fn(batch, tokenizer, max_length=450000):
    """
    Custom collate function for DNA data that converts one-hot encoded sequences to raw sequences
    and tokenizes them.
    
    Args:
        batch: List of tuples where each tuple is (x, y)
               x is one-hot encoded DNA sequence of shape (seq_len, 4)
               y is gene expression data of shape (10, seq_len)
        tokenizer: The GENERator tokenizer
        max_length: Maximum sequence length for tokenization
    
    Returns:
        Dictionary with tokenized inputs and original gene expression data
    """
    # Separate x and y from the batch
    x_batch, y_batch = zip(*batch)
    x_batch = torch.tensor(np.stack(x_batch), dtype=torch.float32)
    y_batch = torch.tensor(np.stack(y_batch), dtype=torch.float32)
    
    # Convert one-hot encoded sequences to raw sequences
    raw_sequences = []
    nucleotides = ['A', 'C', 'G', 'T']
    for one_hot_seq in x_batch:
     
        # Ensure one_hot_seq is a PyTorch tensor
        if not isinstance(one_hot_seq, torch.Tensor):
            one_hot_seq = torch.tensor(one_hot_seq)
        
        # Get indices of 1s in one-hot encoding (argmax along axis 1)
        indices = torch.argmax(one_hot_seq, dim=1).cpu().numpy()
        
        # Convert indices to nucleotides
        raw_seq = ''.join([nucleotides[idx] for idx in indices])
        raw_sequences.append(raw_seq)
    
    # Tokenize the raw sequences
    tokenizer.padding_side = "right"
    inputs = tokenizer(
        raw_sequences,
        add_special_tokens=True,
        return_tensors="pt",
        padding=False,
        truncation=True,
        # max_length=max_length
    )

    
    # Convert y arrays to tensors and stack them
    y_tensors = []
    for y in y_batch:
        if not isinstance(y, torch.Tensor):
            y = torch.tensor(y, dtype=torch.float32)
        y_tensors.append(y)
    
    y_stacked = torch.stack(y_tensors)
    
    # Return tokenized inputs and original y
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "y": y_stacked
    }

In [18]:

# Set logging level for transformers
transformers.logging.set_verbosity_info()

# Define optimization direction for each metric (whether higher or lower is better)
METRICS_DIRECTION: Dict[str, str] = {
    "accuracy": "max",
    "f1_score": "max",
    "mcc": "max",
    "f1_max": "max",
    "auprc_micro": "max",
    "mse": "min",
    "mae": "min",
    "r2": "max",
    "pearson": "max",
}


def is_main_process() -> bool:
    """
    Check if current process is the main process (rank 0) in distributed training.

    Returns:
        bool: True if this is the main process, False otherwise
    """
    if dist.is_initialized():
        return dist.get_rank() == 0
    return True


def dist_print(*args, **kwargs) -> None:
    """
    Print only from the main process (rank 0) in distributed training.
    Prevents duplicate outputs in multi-GPU settings.

    Args:
        *args: Arguments to pass to print function
        **kwargs: Keyword arguments to pass to print function
    """
    if is_main_process():
        print(*args, **kwargs)


In [19]:
def setup_tokenizer(
    model_name: str, padding_and_truncation_side: str
) -> PreTrainedTokenizer:
    """
    Load and configure tokenizer for sequence understanding.

    Args:
        model_name: Name or path of the HuggingFace model
        padding_and_truncation_side: Side for padding and truncation (left or right)

    Returns:
        PreTrainedTokenizer: Configured tokenizer for the model
    """
    dist_print(f"🔤 Loading tokenizer from: {model_name}")
    start_time = time.time()

    # Load tokenizer with trust_remote_code to support custom models
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

    # Configure padding and truncation settings
    tokenizer.padding_side = padding_and_truncation_side
    tokenizer.truncation_side = padding_and_truncation_side

    # Set pad_token to eos_token if not defined
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    dist_print(
        f"⏱️ Tokenizer loading completed in {time.time() - start_time:.2f} seconds"
    )

    return tokenizer

In [20]:
tokenizer = setup_tokenizer("GenerTeam/GENERator-eukaryote-1.2b-base", 'right')

🔤 Loading tokenizer from: GenerTeam/GENERator-eukaryote-1.2b-base


loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/wenduoc/.cache/huggingface/hub/models--GenerTeam--GENERator-eukaryote-1.2b-base/snapshots/3be4abf390afbb7f4d8ccb3370f599338523f1cd/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/wenduoc/.cache/huggingface/hub/models--GenerTeam--GENERator-eukaryote-1.2b-base/snapshots/3be4abf390afbb7f4d8ccb3370f599338523f1cd/tokenizer_config.json
loading file tokenizer.json from cache at None
loading file chat_template.jinja from cache at None


⏱️ Tokenizer loading completed in 0.16 seconds


In [21]:
tokenizer

DNAKmerTokenizer(name_or_path='GenerTeam/GENERator-eukaryote-1.2b-base', vocab_size=4128, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<oov>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("<oov>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [22]:
train_loader2 = DataLoader(
        train_loader.dataset,
        batch_size=4,
        collate_fn=lambda b: collate_fn(b, tokenizer, max_length=196_608)
    )


In [23]:
for batch in train_loader2: 
        print(batch)
        break


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[   1,  289,  291,  ...,  136, 4000,    2],
        [   1, 2088,  236,  ..., 2075, 2303,    2],
        [   1, 4095, 4068,  ...,  151,   51,    2],
        [   1, 2337, 1405,  ..., 3567, 1469,    2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'y': tensor([[[0.1084, 0.1076, 0.0443,  ..., 0.0000, 0.0000, 0.0000],
         [0.1016, 0.0933, 0.0095,  ..., 0.0000, 0.0000, 0.0000],
         [0.1027, 0.1560, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0771, 0.0768, 0.0351,  ..., 0.0000, 0.0193, 0.0000],
         [0.0767, 0.0383, 0.0565,  ..., 0.0000, 0.0000, 0.0000],
         [0.0832, 0.0605, 0.0216,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0402, 0.0616, 0.0000,  ..., 0.0000, 0.0000, 0.4810],
         [0.1345, 0.1501, 0.0000,  ..., 0.0000, 0.9844, 0.0000],
         [0.1726, 0.3892, 0.0251,  ..., 0.0000, 0.0000, 0.0000],
         ...,


In [24]:
batch['input_ids'].shape, batch['y'].shape

(torch.Size([4, 32770]), torch.Size([4, 896, 5313]))

In [25]:
!nvidia-smi

Tue Jul  8 23:09:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:52:00.0 Off |                    0 |
| N/A   30C    P0             60W /  270W |       4MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [26]:
# import math
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from transformers import AutoModel

# class RegulatorySignalPredictor(nn.Module):
#     def __init__(
#         self,
#         base_model_name: str,
#         max_subsequence_length: int = 4096,
#         num_bins: int = 896,
#         num_targets: int = 5313,
#         gradient_checkpointing: bool = True,
#     ):
#         super().__init__()
#         # 1) load your Llama‐style decoder
#         self.base_model = AutoModel.from_pretrained(
#             base_model_name, trust_remote_code=True
#         )
#         if gradient_checkpointing:
#             self.base_model.gradient_checkpointing_enable()

#         self.chunk_size = max_subsequence_length
#         hidden_size = self.base_model.config.hidden_size

#         # 2) adaptive pool along the token dimension → exactly `num_bins`
#         self.pool = nn.AdaptiveAvgPool1d(num_bins)
#         # 3) final head: H → num_targets
#         self.head = nn.Linear(hidden_size, num_targets)

#     def forward(self,
#         input_ids: torch.Tensor,          # [B, 32770]
#         attention_mask: torch.Tensor = None
#     ):
#         B, L = input_ids.shape
#         if attention_mask is None:
#             attention_mask = torch.ones_like(input_ids)

#         # --- chunk & encode ---
#         hidden_chunks = []
#         num_chunks = math.ceil(L / self.chunk_size)
#         for i in range(num_chunks):
#             start = i * self.chunk_size
#             end   = min((i + 1) * self.chunk_size, L)

#             sub_ids  = input_ids[:, start:end]
#             sub_mask = attention_mask[:, start:end]

#             # pad last chunk up to chunk_size so model accepts it
#             if end - start < self.chunk_size:
#                 pad_len = self.chunk_size - (end - start)
#                 sub_ids  = F.pad(sub_ids,  (0, pad_len), value=0)
#                 sub_mask = F.pad(sub_mask, (0, pad_len), value=0)

#             out = self.base_model(
#                 input_ids=sub_ids,
#                 attention_mask=sub_mask
#             )
#             hs = out.last_hidden_state  # → [B, chunk_size, H]

#             # drop the padded tokens
#             real_len = end - start
#             hidden_chunks.append(hs[:, :real_len, :])  # [B, real_len, H]

#         # --- reassemble full sequence hidden states ---
#         x = torch.cat(hidden_chunks, dim=1)    # [B, L, H]

#         # --- pool down to exactly `num_bins` positions ---
#         x = x.transpose(1, 2)                 # [B, H, L]
#         x = self.pool(x)                      # [B, H, num_bins]
#         x = x.transpose(1, 2)                 # [B, num_bins, H]

#         # --- project each bin to your 5,313 signals ---
#         preds = self.head(x)                  # [B, 896, 5313]

#         return {"logits": preds}


In [27]:
# import torch
# import torch.nn as nn
# from transformers import AutoModel

# class RegulatorySignalPredictor(nn.Module):
#     def __init__(
#         self,
#         base_model_name: str,
#         *,
#         max_subsequence_length: int,
#         num_subsequences: int,
#         output_bins: int,
#         output_tracks: int,
#         gradient_checkpointing: bool = True
#     ):
#         super().__init__()
#         self.base_model = AutoModel.from_pretrained(base_model_name, trust_remote_code=True)
#         if gradient_checkpointing:
#             self.base_model.gradient_checkpointing_enable()
        
#         self.max_subsequence_length = max_subsequence_length
#         self.num_subsequences      = num_subsequences
#         self.output_bins           = output_bins
#         self.output_tracks         = output_tracks
        
#         hidden_size = self.base_model.config.hidden_size
#         self.pool = nn.AdaptiveAvgPool1d(self.output_bins)    
#         self.head = nn.Linear(hidden_size, self.output_tracks)  


#     def forward(self, input_ids: torch.LongTensor, attention_mask: torch.LongTensor):
#         # 1) collect all sub‐sequence embeddings
#         seq_chunks = []
#         for i in range(self.num_subsequences):
#             start = i * self.max_subsequence_length
#             end   = (i + 1) * self.max_subsequence_length
            
#             sub_ids  = input_ids[:, start:end]
#             sub_mask = attention_mask[:, start:end]
            
#             out = self.base_model(input_ids=sub_ids, attention_mask=sub_mask)
#             # append the **full** last_hidden_state, not just CLS
#             seq_chunks.append(out.last_hidden_state)  # [B, sub_L, H]
        
#         # 2) stitch back into one long sequence → [B, total_L, H]
#         seq_emb = torch.cat(seq_chunks, dim=1)
        
#         # 3) pool & per‐bin head
#         x = seq_emb.transpose(1, 2)    # → [B, H, total_L]
#         x = self.pool(x)               # → [B, H, output_bins]
#         x = x.transpose(1, 2)          # → [B, output_bins, H]
#         preds = self.head(x)           # → [B, output_bins, output_tracks]
        
#         return {"logits": preds}


In [28]:
import torch
import torch.nn as nn
from transformers import AutoModel

class RegulatorySignalPredictor(nn.Module):
    def __init__(
        self,
        base_model_name: str,
        *,
        max_subsequence_length: int,
        num_subsequences: int,
        output_bins: int,
        output_tracks: int,
        gradient_checkpointing: bool = True
    ):
        super().__init__()
        # 1. backbone
        self.base_model = AutoModel.from_pretrained(base_model_name, trust_remote_code=True)
        if gradient_checkpointing:
            self.base_model.gradient_checkpointing_enable()
        
        # 2. bookkeeping
        self.max_subsequence_length = max_subsequence_length
        self.num_subsequences    = num_subsequences
        self.output_bins         = output_bins
        self.output_tracks       = output_tracks
        
        # 3. pooling + small head (instead of one giant regression matrix)
        hidden_size = self.base_model.config.hidden_size
   
        # self.head = nn.Linear(hidden_size, self.output_tracks) 
        self.head = nn.Sequential(
            nn.Linear(hidden_size * num_subsequences, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, output_bins * output_tracks),
        )


    def forward(self, input_ids: torch.LongTensor, attention_mask: torch.LongTensor):
        """
        input_ids:       [B, L]
        attention_mask:  [B, L]
        returns preds:   [B, output_bins, output_tracks]
        """
        # 1) run each chunk through the backbone and collect all token embeddings
        hidden_states = []
        for i in range(self.num_subsequences):
            start_idx = i * self.max_subsequence_length
            end_idx = (i + 1) * self.max_subsequence_length
            sub_input_ids = input_ids[:, start_idx:end_idx]
            sub_attention_mask = attention_mask[:, start_idx:end_idx]

            outputs = self.base_model(input_ids=sub_input_ids, attention_mask=sub_attention_mask)
            last_hidden_state = outputs.last_hidden_state
            cls_embedding = last_hidden_state[:, -1, :]
            hidden_states.append(cls_embedding)
        
        # 2) stitch back into one long sequence → [B, total_L, H]
        combined_hidden_states = torch.cat(hidden_states, dim=-1) # [4, 16384]
        
        
        preds = self.head(combined_hidden_states )
        
        return {"logits": preds}

In [29]:
model = RegulatorySignalPredictor(
    base_model_name="GenerTeam/GENERator-eukaryote-1.2b-base",
    max_subsequence_length=4096,   # e.g. 32770/8 ≈ 4096
    num_subsequences=8,
    output_bins=896,
    output_tracks=5313
)


loading configuration file config.json from cache at /home/wenduoc/.cache/huggingface/hub/models--GenerTeam--GENERator-eukaryote-1.2b-base/snapshots/3be4abf390afbb7f4d8ccb3370f599338523f1cd/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 16384,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 26,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3",
  "use_cache": true,
  "vocab_size": 4128
}

loading weights file model.safetensors from cache at /home/wenduoc/.cache/huggingface/hub/models-

In [30]:
model=model.to(torch.bfloat16).to(device)

In [31]:
with torch.no_grad():
    preds = model(batch['input_ids'].to(device), batch['attention_mask'].to(device))
preds

{'logits': tensor([[ 1.4844, -2.4219,  3.1562,  ..., -0.5430, -2.2344,  0.3613],
         [ 2.3281, -2.0312,  3.5469,  ..., -1.2656, -1.7891,  0.0505],
         [ 2.0469, -1.8828,  3.5469,  ..., -0.7109, -1.9688,  0.1641],
         [ 1.2500, -2.3750,  2.4531,  ..., -1.1797, -1.4375,  0.2314]],
        device='cuda:0', dtype=torch.bfloat16)}

In [32]:
model

RegulatorySignalPredictor(
  (base_model): LlamaModel(
    (embed_tokens): Embedding(4128, 2048)
    (layers): ModuleList(
      (0-25): 26 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (

In [33]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Total parameters:     {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Trainable %:          {100 * trainable_params / total_params:.2f}%")


In [34]:
count_parameters(model)

Total parameters:     2,381,237,376
Trainable parameters: 2,381,237,376
Trainable %:          100.00%


In [35]:
for param in model.base_model.parameters():
    param.requires_grad = False

# Unfreeze the last 8 LLaMA decoder layers
for layer in model.base_model.layers[-6:]:
    for param in layer.parameters():
        param.requires_grad = True


In [36]:
count_parameters(model)

Total parameters:     2,381,237,376
Trainable parameters: 1,491,895,424
Trainable %:          62.65%


# Train

In [37]:

# def evaluate_model_custom(
#     model: PreTrainedModel,
#     data_loader: DataLoader,
#     device: str,
#     criterion: nn.Module
# ) -> Dict[str, float]:
#     """
#     Evaluate regression model on a dataset.
    
#     Args:
#         model: Model to evaluate
#         data_loader: DataLoader for evaluation
#         device: Device to run evaluation on
#         criterion: Loss function (MSE)
    
#     Returns:
#         Dictionary of evaluation metrics (MSE, MAE, R², PCC)
#     """
#     model.eval()
#     total_loss = 0.0
#     all_predictions = []
#     all_labels = []
#     num_batches = 0
    
#     with torch.no_grad():
#         for batch in tqdm(data_loader, desc="Evaluating"):
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['y'].to(device)

#             # input_ids = input_ids.to(model.dtype)
#             # attention_mask = attention_mask.to(model.dtype)
            
#             outputs = model(input_ids=input_ids,attention_mask=attention_mask)
#             logits = outputs['logits']
            
#             # Process labels for regression
#             if labels.dim() > 1:
#                 labels = labels.view(-1).float()
            
#             # Calculate MSE loss
#             loss = criterion(logits.view(-1), labels)
            
#             # Get predictions (no activation function needed for regression)
#             predictions = logits.float().cpu().numpy()
     
#             total_loss += loss.item()
            
#             # Store for metrics
#             all_predictions.extend(predictions.flatten())
#             all_labels.extend(labels.cpu().numpy().flatten())
                
#             num_batches += 1
    
#     # Compute average loss
#     avg_loss = total_loss / num_batches
    
#     # Convert lists to numpy arrays
#     all_predictions = np.array(all_predictions)
#     all_labels = np.array(all_labels)
    
#     # Mean Squared Error
#     mse = mean_squared_error(all_labels, all_predictions)
    
#     # Mean Absolute Error
#     mae = mean_absolute_error(all_labels, all_predictions)
    
#     # R-squared score
#     r2 = r2_score(all_labels, all_predictions)
    
#     # Pearson Correlation Coefficient
#     pcc, p_value = pearsonr(all_labels, all_predictions)
    
#     # Root Mean Squared Error
#     rmse = np.sqrt(mse)
    
#     return {
#         'loss': avg_loss,
#         'mse': mse,
#         'mae': mae,
#         'rmse': rmse,
#         'r2': r2,
#         'pcc': pcc,
#         'pcc_p_value': p_value,
#         'num_samples': len(all_predictions)
#     }

In [40]:
from tqdm import tqdm
import torch
import torch.nn as nn
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import os
from typing import Dict, Any, Optional
from torch.utils.data import DataLoader
from transformers import PreTrainedModel, PreTrainedTokenizer



import math
import torch
from tqdm import tqdm
from typing import Dict
from scipy.stats import t as t_dist

def evaluate_model_custom(
    model: torch.nn.Module,
    data_loader: torch.utils.data.DataLoader,
    device: str,
    criterion: torch.nn.Module
) -> Dict[str, float]:
    """
    Streaming evaluation: regression metrics (MSE, MAE, RMSE, R², PCC)
    entirely in PyTorch to avoid NumPy import issues.
    """
    model.eval()

    # Streaming accumulators (all Python floats)
    n = 0
    sum_loss     = 0.0
    sum_sq_err   = 0.0
    sum_abs_err  = 0.0
    sum_pred     = 0.0
    sum_lbl      = 0.0
    sum_pred_sq  = 0.0
    sum_lbl_sq   = 0.0
    sum_pred_lbl = 0.0

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids      = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels         = batch['y'].view(-1).float().to(device)

            outputs  = model(input_ids=input_ids, attention_mask=attention_mask)
            preds_t  = outputs['logits'].view(-1)  # might be bfloat16
            preds_f  = preds_t.float()             # cast to float32
            b        = preds_f.size(0)
            n       += b

            # loss
            loss_b = criterion(preds_t, labels).item()
            sum_loss += loss_b * b

            # streaming stats in torch
            diff      = preds_f - labels
            sum_sq_err   += diff.pow(2).sum().item()
            sum_abs_err  += diff.abs().sum().item()
            sum_pred     += preds_f.sum().item()
            sum_lbl      += labels.sum().item()
            sum_pred_sq  += preds_f.pow(2).sum().item()
            sum_lbl_sq   += labels.pow(2).sum().item()
            sum_pred_lbl += (preds_f * labels).sum().item()

    # compute final metrics
    mse  = sum_sq_err / n
    mae  = sum_abs_err / n
    rmse = math.sqrt(mse)

    ss_tot = sum_lbl_sq - (sum_lbl**2) / n
    r2     = 1 - (sum_sq_err / ss_tot) if ss_tot > 0 else float('nan')

    num = n * sum_pred_lbl - sum_pred * sum_lbl
    den = math.sqrt((n * sum_pred_sq - sum_pred**2) * (n * sum_lbl_sq - sum_lbl**2))
    pcc = num / den if den > 0 else 0.0

    if n > 2:
        t_stat = pcc * math.sqrt((n - 2) / (1 - pcc**2))
        p_val  = 2 * (1 - t_dist.cdf(abs(t_stat), df=n-2))
    else:
        p_val = float('nan')

    return {
        'loss':        sum_loss / n,
        'mse':         mse,
        'mae':         mae,
        'rmse':        rmse,
        'r2':          r2,
        'pcc':         pcc,
        'pcc_p_value': p_val,
        'num_samples': n
    }


def train_model_custom(
    model: PreTrainedModel,
    tokenizer: PreTrainedTokenizer,
    train_loader: DataLoader,
    val_loader: DataLoader,
    test_loader: Optional[DataLoader] = None,
    num_epochs: int = 10,
    learning_rate: float = 1e-5,
    weight_decay: float = 0.01,
    warmup_steps: int = 0,
    max_grad_norm: float = 1.0,
    save_dir: str = None,
    save_steps: int = 10,
    early_stopping_patience: int = 5,
    device: str = "cuda",
    use_wandb: bool = False,
    gradient_accumulation_steps: int = 1,

) -> Dict[str, Any]:
    """
    Custom training function for DNA sequence models with gene expression prediction (regression).
    Modified to evaluate after each epoch and save model based on lowest validation loss (MSE).
    """
    model = model.to(device)
    model.train()

    if save_dir is None:
        save_dir = f"/work/magroup/wenduoc/DNALongBench/experiments/GENERator/results/RSAP/human/v2"
    os.makedirs(save_dir, exist_ok=True)
    
    train_loader_custom = DataLoader(
        train_loader.dataset,
        batch_size=4,
        collate_fn=lambda b: collate_fn(b, tokenizer)
    )

    val_loader_custom = DataLoader(
        val_loader.dataset,
        batch_size=4,
        collate_fn=lambda b: collate_fn(b, tokenizer)
    )

    if test_loader is not None:
        test_loader_custom = DataLoader(
            test_loader.dataset,
            batch_size=4,
            collate_fn=lambda b: collate_fn(b, tokenizer)
        )

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )

    # scheduler = torch.optim.lr_scheduler.LinearLR(
    #     optimizer,
    #     start_factor=0.1,
    #     end_factor=1.0,
    #     total_iters=warmup_steps
    # )
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        patience=1,
        factor=0.5,
        mode='min'
    )

    # Use MSE loss for regression
    criterion = nn.MSELoss()

    best_val_loss = float('inf')
    best_val_pcc = -1.0  # Track best PCC as well
   
    global_step = 0
    training_history = {
        'train_loss': [],
        'val_loss': [],
        'val_pcc': [],
        'val_r2': [],
        'learning_rates': []
    }

    print(f"🚀 Starting regression training for {num_epochs} epochs...")
    print(f"🔧 Gradient accumulation steps: {gradient_accumulation_steps}")
    print(f"📊 Evaluation will occur after each epoch")
    print(f"💾 Model will be saved based on lowest validation MSE loss")

    start_time = time.time()

    best_train_loss = float('inf')
    for epoch in range(num_epochs):
        print(f"\n{'='*50}")
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"{'='*50}")

        model.train()
        epoch_train_loss = 0.0
        train_steps = 0

        progress_bar = tqdm(train_loader_custom, desc=f"Epoch {epoch + 1}")

        for step, batch in enumerate(progress_bar):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['y'].to(device)

            # input_ids = input_ids.to(model.dtype)
            # attention_mask = attention_mask.to(model.dtype)

            with torch.cuda.amp.autocast():
                outputs = model(input_ids=input_ids,attention_mask=attention_mask)
                logits = outputs['logits']

                if labels.dim() > 1:
                    labels = labels.view(-1).float()

                # MSE loss for regression
                loss = criterion(logits.view(-1), labels)
                loss = loss / gradient_accumulation_steps

                step_train_loss = loss.item() * gradient_accumulation_steps

            loss.backward()

            if (step + 1) % gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                optimizer.step()
                # scheduler.step()
                optimizer.zero_grad()
                global_step += 1

            epoch_train_loss += loss.item() * gradient_accumulation_steps
            train_steps += 1

            progress_bar.set_postfix({
                'mse_loss': f"{(loss.item() * gradient_accumulation_steps):.4f}",
                'lr': f"{scheduler.get_last_lr()[0]:.2e}"
            })

            # if global_step % save_steps == 0:
            if global_step > 0 and global_step % 50 == 0:
                checkpoint_path = os.path.join(save_dir, f'checkpoint_step_{global_step}.pt')
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'global_step': global_step
                }, checkpoint_path)
                print(f"💾 Intermediate checkpoint saved at step {global_step}")
            if step_train_loss < best_train_loss:
                best_train_loss = step_train_loss
                checkpoint_path = os.path.join(save_dir, f'best_model.pt')
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'global_step': global_step
                }, checkpoint_path)
                print(f"💾 [step {global_step}] new best TRAIN loss: {best_train_loss:.4f}")
                # else:
                #     print(f"⏭️ [step {global_step}] train loss {step_train_loss:.4f} ≥ best {best_train_loss:.4f}, skipping save")
         

        avg_train_loss = epoch_train_loss / train_steps
        training_history['train_loss'].append(avg_train_loss)
        training_history['learning_rates'].append(scheduler.get_last_lr()[0])

        print(f"\n🔄 Evaluating after epoch {epoch + 1}...")
        val_metrics = evaluate_model_custom(model, val_loader_custom, device, criterion)
        training_history['val_loss'].append(val_metrics['loss'])
        training_history['val_pcc'].append(val_metrics['pcc'])
        training_history['val_r2'].append(val_metrics['r2'])

        print(f"\n📈 Epoch {epoch + 1} Summary:")
        print(f"  Train MSE Loss: {avg_train_loss:.4f}")
        print(f"  Val MSE Loss: {val_metrics['loss']:.4f}")
        print(f"  Val PCC: {val_metrics['pcc']:.4f}")
        print(f"  Val R²: {val_metrics['r2']:.4f}")
        print(f"  Val RMSE: {val_metrics['rmse']:.4f}")
        print(f"  Learning Rate: {scheduler.get_last_lr()[0]:.2e}")

        # Save model based on lowest validation loss
        if val_metrics['loss'] < best_val_loss:
            best_val_loss = val_metrics['loss']
            best_val_pcc = val_metrics['pcc']

            scheduler.step(val_metrics['loss'])

            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_val_loss': best_val_loss,
                'best_val_pcc': best_val_pcc,
                'global_step': global_step
            }, os.path.join(save_dir, 'best_model.pt'))

            print(f"💾 New best model saved! Val MSE: {best_val_loss:.4f}, Val PCC: {best_val_pcc:.4f}")
       
        epoch_checkpoint_path = os.path.join(save_dir, f'model_epoch_{epoch + 1}.pt')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_loss': val_metrics['loss'],
            'val_pcc': val_metrics['pcc'],
            'global_step': global_step
        }, epoch_checkpoint_path)
        print(f"💾 Epoch {epoch + 1} checkpoint saved")

    
        model.train()

    total_time = time.time() - start_time
    print(f"\n✅ Training completed in {total_time/60:.2f} minutes")
    print(f"🏆 Best validation MSE loss achieved: {best_val_loss:.4f}")
    print(f"🏆 Best validation PCC achieved: {best_val_pcc:.4f}")

    final_metrics = {
        'training_history': training_history, 
        'best_val_loss': best_val_loss,
        'best_val_pcc': best_val_pcc
    }
    
    if test_loader is not None:
        print("\n🧪 Evaluating on test set...")
        test_metrics = evaluate_model_custom(model, test_loader_custom, device, criterion)
        final_metrics['test_metrics'] = test_metrics

        print("📊 Final Test Metrics:")
        print(f"  MSE: {test_metrics['mse']:.4f}")
        print(f"  RMSE: {test_metrics['rmse']:.4f}")
        print(f"  MAE: {test_metrics['mae']:.4f}")
        print(f"  R²: {test_metrics['r2']:.4f}")
        print(f"  PCC: {test_metrics['pcc']:.4f}")
        print(f"  PCC p-value: {test_metrics['pcc_p_value']:.6f}")

    return final_metrics

In [41]:


# Train the model
training_results = train_model_custom(
    model=model,
    tokenizer=tokenizer,
    train_loader=train_loader,
    val_loader=valid_loader,
    test_loader=test_loader,
    num_epochs=3,
    learning_rate=1e-5,
    # batch_size=1,  # Start small due to memory constraints

    device=device,
    use_wandb=False,  
    gradient_accumulation_steps=16,  # Effective batch size = 1 * 8 = 8
)



🚀 Starting regression training for 3 epochs...
🔧 Gradient accumulation steps: 16
📊 Evaluation will occur after each epoch
💾 Model will be saved based on lowest validation MSE loss

Epoch 1/3


Epoch 1: 0it [00:00, ?it/s]/tmp/ipykernel_2719257/1919700480.py:210: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1: 1it [00:25, 25.36s/it, mse_loss=14.4632, lr=1.00e-05]

💾 [step 0] new best TRAIN loss: 14.4632


Epoch 1: 2it [00:50, 24.97s/it, mse_loss=10.3192, lr=1.00e-05]

💾 [step 0] new best TRAIN loss: 10.3192


...

💾 Intermediate checkpoint saved at step 350


Epoch 1: 5615it [25:07:01, 50.10s/it, mse_loss=6.2878, lr=1.00e-05]

💾 Intermediate checkpoint saved at step 350


Epoch 1: 5677it [25:22:49, 16.09s/it, mse_loss=3.6801, lr=1.00e-05] 


KeyboardInterrupt: 

In [42]:
checkpoint = torch.load(f"/work/magroup/wenduoc/DNALongBench/experiments/GENERator/results/RSAP/human/v2/best_model.pt", map_location=device)

model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
import math
import torch
from tqdm import tqdm
from typing import Dict
from scipy.stats import t as t_dist

def evaluate_model_custom(
    model: torch.nn.Module,
    data_loader: torch.utils.data.DataLoader,
    device: str,
    criterion: torch.nn.Module
) -> Dict[str, float]:
    """
    Streaming evaluation: regression metrics (MSE, MAE, RMSE, R², PCC)
    entirely in PyTorch to avoid NumPy import issues.
    """
    model.eval()

    # Streaming accumulators (all Python floats)
    n = 0
    sum_loss     = 0.0
    sum_sq_err   = 0.0
    sum_abs_err  = 0.0
    sum_pred     = 0.0
    sum_lbl      = 0.0
    sum_pred_sq  = 0.0
    sum_lbl_sq   = 0.0
    sum_pred_lbl = 0.0

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids      = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels         = batch['y'].view(-1).float().to(device)

            outputs  = model(input_ids=input_ids, attention_mask=attention_mask)
            preds_t  = outputs['logits'].view(-1)  # might be bfloat16
            preds_f  = preds_t.float()             # cast to float32
            b        = preds_f.size(0)
            n       += b

            # loss
            loss_b = criterion(preds_t, labels).item()
            sum_loss += loss_b * b

            # streaming stats in torch
            diff      = preds_f - labels
            sum_sq_err   += diff.pow(2).sum().item()
            sum_abs_err  += diff.abs().sum().item()
            sum_pred     += preds_f.sum().item()
            sum_lbl      += labels.sum().item()
            sum_pred_sq  += preds_f.pow(2).sum().item()
            sum_lbl_sq   += labels.pow(2).sum().item()
            sum_pred_lbl += (preds_f * labels).sum().item()

    # compute final metrics
    mse  = sum_sq_err / n
    mae  = sum_abs_err / n
    rmse = math.sqrt(mse)

    ss_tot = sum_lbl_sq - (sum_lbl**2) / n
    r2     = 1 - (sum_sq_err / ss_tot) if ss_tot > 0 else float('nan')

    num = n * sum_pred_lbl - sum_pred * sum_lbl
    den = math.sqrt((n * sum_pred_sq - sum_pred**2) * (n * sum_lbl_sq - sum_lbl**2))
    pcc = num / den if den > 0 else 0.0

    if n > 2:
        t_stat = pcc * math.sqrt((n - 2) / (1 - pcc**2))
        p_val  = 2 * (1 - t_dist.cdf(abs(t_stat), df=n-2))
    else:
        p_val = float('nan')

    return {
        'loss':        sum_loss / n,
        'mse':         mse,
        'mae':         mae,
        'rmse':        rmse,
        'r2':          r2,
        'pcc':         pcc,
        'pcc_p_value': p_val,
        'num_samples': n
    }


In [43]:
# Final evaluation on test set if provided
final_metrics = {}

test_loader_custom = DataLoader(
            test_loader.dataset,
            batch_size=4,
            collate_fn=lambda b: collate_fn(b, tokenizer)
        )

criterion = nn.MSELoss()

print("\n🧪 Evaluating on test set...")
test_metrics = evaluate_model_custom(model, test_loader_custom, device, criterion)
final_metrics['test_metrics'] = test_metrics

print("📊 Final Test Metrics:")
for key, value in test_metrics.items():
    print(f"  {key}: {value:.4f}")

print(final_metrics)


🧪 Evaluating on test set...


Evaluating: 484it [2:06:54, 15.76s/it]2025-07-10 02:42:25.494796: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Evaluating: 485it [2:06:58, 15.71s/it]

📊 Final Test Metrics:
  loss: 7.9204
  mse: 7.9204
  mae: 1.0208
  rmse: 2.8143
  r2: -0.1476
  pcc: 0.0000
  pcc_p_value: 0.0459
  num_samples: 9220987776.0000
{'test_metrics': {'loss': 7.920360825144266, 'mse': 7.920361022502238, 'mae': 1.0207566303794653, 'rmse': 2.8143135970432005, 'r2': -0.14755374438828572, 'pcc': 2.0789503475330543e-05, 'pcc_p_value': 0.04589776118305866, 'num_samples': 9220987776}}
